In [20]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [5]:
def get_source(url):
    """Return the source code of the provided url.
    
    Args: 
        url (string): URL of the page to scrape.
        
    Returns:
        response (object): HTTP response object from requests_html
    """  
    
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    
    except requests.exceptions.RequestException as e:
        print(e)

In [9]:
def scrape_google(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    response1 = get_source("https://www.google.com/search?q=" + query + '&start=10')
    
    links = list(response.html.absolute_links)
    links += list(response1.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    
    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
    
    return links

In [60]:
url = "https://www.betterup.com/blog/work-life-balance"

In [ ]:
r1 = requests.get(url)